In [1]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
from cftime import datetime 
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
jet = mcm.get_cmap('jet')
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
%matplotlib inline 


In [2]:
#Creates timestamps for 28 consecutive days (array length 672)

day = ['00000','03600', '07200', '10800', '14400', '18000',
 '21600', '25200', '28800', '32400', '36000', '39600', 
 '43200', '46800', '50400', '54000', '57600', '61200', 
 '64800', '68400', '72000', '75600', '79200', '82800']

day2 = np.stack((day, day, day, day, day, day, day), axis=0)
day2 = np.reshape(day2, (24*7))
day3 = np.stack((day2, day2, day2, day2), axis=0)
day3 = np.reshape(day3, (168*4))
day3.shape

(672,)

In [3]:
# Creates an array (list) of date + time strings in the format '2003-02-01-03600'
# To be used in a loop to identify file names to open one at a time below

times = xr.cftime_range(start="0001-12-01", periods=672, freq="1H", calendar="noleap")
#times2 = times.strftime("%Y%m%d %H:%M")
#times3 = ds["time"].dt.strftime("%Y%m%d")

timear = np.arange(0,672)
times_dt = []
for t in timear:
    tempstr = str(times[t])
    tempstr2 = tempstr[0:10]
    times_dt.append( tempstr2 + '-' + day3[t] )
    
#times_dt

In [5]:
# Opens first file (named)
# Drops unnecessary variables

#filename1="hist/cesm220_FXSD1_f19_f19_mg17_arc4.cam.h1.2003-02-01-00000.nc" #from arc4 nobackup
filename1="Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-12-01-00000.nc" 

ds1 = xr.open_dataset(filename1)
ds1

<xarray.Dataset>
Dimensions:       (lat: 96, lon: 144, mlat: 97, mlon: 80, lev: 126, ilev: 127, time: 24, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon           (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * mlat          (mlat) float64 -90.0 -88.12 -86.24 -84.33 ... 86.24 88.12 90.0
  * mlon          (mlon) float64 -180.0 -175.5 -171.0 ... 166.5 171.0 175.5
  * lev           (lev) float64 4.055e-10 5.207e-10 6.686e-10 ... 970.6 992.6
  * ilev          (ilev) float64 3.551e-10 4.559e-10 5.854e-10 ... 985.1 1e+03
  * time          (time) object 0001-12-01 00:00:00 ... 0001-12-01 23:00:00
Dimensions without coordinates: nbnd
Data variables: (12/113)
    gw            (lat) float64 ...
    hyam          (lev) float64 ...
    hybm          (lev) float64 ...
    P0            float64 ...
    hyai          (ilev) float64 ...
    hybi          (ilev) float64 ...
    ...            ...
    UI            (time, lev, lat, lon) float32 ...
    V             (time, lev, lat, lon) float32 ...
    VI            (time, lev, lat, lon) float32 ...
    WI            (time, lev, lat, lon) float32 ...
    Z3            (time, lev, lat, lon) float32 ...
    e             (time, lev, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport
    logname:           earfw
    host:              d13s7b4.arc4.lee
    initial_file:      /nobackup/earfw/cesm2/cases/ACP_CESM213_FX2000_f19_f19...
    topography_file:   /nobackup/earfw/cesm2_inputdata//atm/cam/topo/USGS-gto...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  hour_1

In [6]:
ds1_dr = ds1.drop_vars({'gw','ndbase','nsbase','nbdate','nbsec','mdt','ndcur',
                    'nscur','co2vmr','ch4vmr','n2ovmr','f11vmr','f12vmr',
                    'sol_tsi','f107','f107a','f107p','kp','ap','colat_crit1',
                    'colat_crit2','nsteph','CO2','Fe2O2H2','FeN2p','FeO','FeO2'
                    ,'FeO2H2','FeO2p','FeO3','FeOH','FeOp','H','H2','H2O','HO2'
                    ,'NO','NOp','Na2H2C2O6','NaCO2p','NaH2Op','NaHCO3','NaN2p'
                    ,'NaO','NaO2','NaOH','NaOp','O','O2','O2_1D','O2p','O3','OH','SZA'
                    ,'date_written','time_written','Na','Nap','OMEGA','U','V'
                    #, 'T', 'Z3', 'e'
                       })
#,'Date','Datesec'
#ds1_dr

ValueError: One or more of the specified variables cannot be found in this dataset

In [12]:
# Not enough memory to open and process all the time files in one loop so had to do it in chunks / multiple arrays and then do this and the below cell multiple times

#Whole array
#timear2 = np.arange(1,672)

#24h
#timear = np.arange(1,26) 

# 7 days split into two chunks
timear1 = np.arange(1,49) 
#timear2 = np.arange(96,168) 

#print(timear11.size)
print(timear11[-1])

48


In [14]:
# Open each file in turn, drop unnecessary variables and then concatenate to ds1_dr dataset 

for t in timear1:
    temp_filename="Nc_Files/Wuhu_WACCMX_files/cesm220_FXSD1_f19_f19_mg17_arc4.cam.h1." + times_dt[t] + ".nc"
    temp_ds = xr.open_dataset(temp_filename)
    temp_ds_dr = temp_ds.drop_vars({'gw','ndbase','nsbase','nbdate','nbsec','mdt','ndcur',
                    'nscur','co2vmr','ch4vmr','n2ovmr','f11vmr','f12vmr',
                    'sol_tsi','f107','f107a','f107p','kp','ap','colat_crit1',
                    'colat_crit2','nsteph','CO2','Fe2O2H2','FeN2p','FeO','FeO2'
                    ,'FeO2H2','FeO2p','FeO3','FeOH','FeOp','H','H2','H2O','HO2'
                    ,'NO','NOp','Na2H2C2O6','NaCO2p','NaH2Op','NaHCO3','NaN2p'
                    ,'NaO','NaO2','NaOH','NaOp','O','O2','O2_1D','O2p','O3','OH','SZA'
                    ,'date_written','time_written','Na','Nap','OMEGA','U','V'
                    #, 'T', 'Z3', 'e'
                            })
    ds1_dr = xr.concat((ds1_dr,temp_ds_dr), 'time')

In [15]:
# Check last element in the time array of ds1_dr to make sure the above loop completed ok

ds1_dr.time[-1] 

#47 2003-02-02T23:00:00
#48 2003-02-03T00:00:00
#49 2003-02-03T01:00:00

<xarray.DataArray 'time' ()>
array('2003-02-03T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2003-02-03
Attributes:
    long_name:  time
    bounds:     time_bnds

In [20]:
# Save dataset to a netcdf file

#24h
#ds1_dr.to_netcdf(path="Wuhu_WACCMX_run_Feb_day1.ncdf", format='NETCDF3_64BIT')

#48h
ds1_dr.to_netcdf(path="Nc_Files/Wuhu_WACCMX_files/Wuhu_WACCMX_run_Feb_day1-2.ncdf", format='NETCDF3_64BIT')

/localhome/sestay/miniconda3/lib/python3.9/site-packages/xarray/conventions.py:441: UserWarning: Variable 'time' has datetime type and a bounds variable but time.encoding does not have units specified. The units encodings for 'time' and 'time_bnds' will be determined independently and may not be equal, counter to CF-conventions. If this is a concern, specify a units encoding for 'time' before writing to a file.
  warnings.warn(
